In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid
import ccxt

In [ ]:
print(ccxt.exchanges) # print a list of all available exchange classes

In [ ]:
import ccxt
polo = ccxt.poloniex({
    'apiKey': cfg.POLONIEX_API_KEY,
    'secret': cfg.POLONIEX_API_SECRET_KEY,
})
gdax = ccxt.gdax({
    'apiKey': cfg.GDAX_API_KEY,
    'secret': cfg.GDAX_API_SECRET_KEY,
    'passphrase': cfg.GDAX_PASSPHRASE
})
binance = ccxt.binance()

exchange = polo
markets = exchange.load_markets()

print(exchange.id, markets)
print(exchange.fetch_order_book(exchange.symbols[0]))
print(exchange.fetch_ticker('ETH/BTC')) # coin/market - base/quote
print(exchange.fetch_trades('LTC/BTC'))

In [ ]:
# Available methods
print (dir (ccxt.poloniex ()))

In [ ]:
# Asyncronous
# import asyncio
# import ccxt.async as ccxt

async def print_poloniex_ethbtc_ticker():
    print(await exchange.fetch_ticker('ETH/BTC'))

asyncio.get_event_loop().run_until_complete(print_poloniex_ethbtc_ticker())

In [ ]:
# Prices
def get_ohlcv_fpath(coin, market, exchange_id, period):
    fname = '{:s}_{:s}_{:s}_{:s}.csv'.format(
        exchange_id, coin, market, str(period))
    return os.path.join(cfg.DATA_DIR, fname)

def get_symbol(coin, market):
    return coin+'/'+market

def fetch_chart_data(exchange, coin, market, period, start, end):
    assert period in exchange.timeframes
    data = exchange.fetch_ohlcv(get_symbol(coin, market), period)#, start, end)
    df = pd.DataFrame(data, columns = c.OHLCV_COLUMNS)
    df['time'] = [datetime.datetime.fromtimestamp(t//1000) for t in df['time']]
    df.set_index('time', inplace=True)
    return df    

def fetch_and_save_chart_data(exchange, coin, market, period, start, end):
    print("Downloading:", get_symbol(coin, market))
    df = fetch_chart_data(exchange, coin, market, period, start, end)
    fpath = get_ohlcv_fpath(coin, market, exchange.id, period)
    df.to_csv(fpath, index=True)
    print("Downloaded rows:", len(df))
    return df

def load_chart_data(exchange_id, coin, market, period):
    fpath = get_ohlcv_fpath(coin, market, exchange_id, period)
    df = pd.read_csv(fpath, parse_dates=True, index_col='time')
    return df

def download_ohlcv(exchange, coins, market, period, start, end):
    for coin in coins:
        _ = fetch_and_save_chart_data(exchange, coin, market, period, start, end)

def load_currency_group_prices(exchange_id, coins, market, period, start, end):
    df = pd.DataFrame()
    for coin in coins:
        symbol = get_symbol(coin, market)
        data = load_chart_data(exchange_id, coin, market, period)
        data = data[(data.index >= start) & (data.index <= end)]
        df[symbol] = data['close']
    df.dropna(inplace=True)
    return df

In [ ]:
# Get historical data
coins = [coins.BTC, coins.LTC, coins.ETH, coins.XRP, coins.XMR, c.DASH]
market = coins.USDT
coin = coins.LTC
symbol = get_symbol(coin, market)
period = '30m' #1800
start = datetime.datetime(year=2015, month=9, day=1)
end = datetime.datetime(year=2018, month=1, day=1)

In [ ]:
#df = fetch_chart_data(exchange, coin, market, period, start, end)
#df = fetch_and_save_chart_data(exchange, coin, market, period, start, end)
df = load_chart_data(exchange.id, coin, market, period)
df.head()

In [ ]:
download_ohlcv(exchange, coins, market, period, start, end)

In [ ]:
df = load_currency_group_prices(exchange.id, coins, market, period, start, end)
df.head()

In [ ]:
# Trading
print(exchange.fetch_balance())

# sell one ฿ for market price and receive $ right now
print(exchange.id, exchange.create_market_sell_order('BTC/USD', 1))

# limit buy BTC/EUR, you pay €2500 and receive ฿1  when the order is closed
print(exchange.id, exchange.create_limit_buy_order('BTC/EUR', 1, 2500.00))

# pass/redefine custom exchange-specific order params: type, amount, price, flags, etc...
polo.create_market_buy_order('BTC/USD', 1, {'trading_agreement': 'agree'})